
# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shap
import pickle

from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore", message="Empty dataset at worker")

# Loading data
Data consists of all PL players logs for 2021-22 and 2022-23 seasons\
A single log is just a summary of player performance in a particular match

In [3]:
# loading csv
df_original = pd.read_csv('data/previous/FPL_logs.csv')

df = df_original.copy()

In [4]:
df_original.columns

Index(['Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name', 'Start', 'Pos',
       'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR',
       'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA', 'SCA', 'GCA',
       'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'T_att', 'T_succ',
       'Season', 'WDL', 'Team_score', 'Opp_score', 'Team_CS', 'Was_home', 'GW',
       'Clean_name', 'FPL_name', 'FPL_team', 'FPL_opp_team', 'Name_original',
       'FPL_pos', 'FPL_ast', 'Bonus', 'Bps', 'Player_CS', 'Creativity',
       'Element', 'Fixture', 'Player_GC', 'Goals_scored', 'ICT_index',
       'Influence', 'Kickoff_time', 'FPL_min', 'Own_goals', 'Penalties_missed',
       'Penalties_saved', 'Saves', 'Selected', 'Threat', 'FPL_points',
       'Transfers_balance', 'Transfers_in', 'Transfers_out', 'Value', 'FPL_GW',
       'xP', 'FPL_xA', 'FPL_xGI', 'FPL_xG', 'FPL_xGC', 'Opp_rating',
       'Team_rating', 'Min_points'],
      dtype='object')

In [5]:
df_original["Name"].unique().size

484

In [6]:
df.shape

(19421, 80)

In [7]:
df["Season"].unique()

array(['2021-22', '2022-23', '2023-24'], dtype=object)

In [8]:
# df = df[~df["FPL_pos"].isin(["GK"])]

In [9]:
df["Season"].unique()

array(['2021-22', '2022-23', '2023-24'], dtype=object)

In [10]:
df = df[df["Season"].isin(['2021-22', '2022-23'])]
df = df[~~df["FPL_pos"].isin(['GK'])]

In [11]:
df = df.sort_values(by=["Name", "Date"])

# Feature selection
Model will use a set of standard features and moving averages for some other feature values. In this case we will use moving averages for last 2, 4 and 30 gameweeks.

In [12]:
rolling_gameweeks = [2, 4, 30]
# rolling_gameweeks = [5]

# to_predict = "FPL_points"
to_predict = "xP"
standard_features = [
             "Was_home", # Home/Away
             # "Team_rating", "Opp_rating", 
             "Rating_difference" # Team ratings
             ]
features_to_roll = [
                "Min", 
                # "Start", # time played
                'Gls', 
                # 'Sh', 'SoT', # Goals
                'Ast', # Assists
                # 'CrdY', 'CrdR', # Cards
                "xG", 'xA', # Expected
                'Team_CS', # Defence
                'Team_score', 'Opp_score', 'Team_result', # Team form
                # "xGPoints", "CSPoints", # Position-scaled
                # "Cmp%", "PrgP", "PrgC", "T_succ",
                # 'bonus', 'bps', # Bonus
                # 'ICT_index', # ICT
                "FPL_points", 
                # "Baseline_points", "Bonus" # FPL points
            ]
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "FPL_points"]

In [13]:
# df[df["Name"] == "Erling-Haaland"]

# Feature engineering

In [14]:
df["Rating_difference"] = df["Team_rating"] / df["Opp_rating"]
df["Baseline_points"] = df["FPL_points"] - df["Bonus"]

In [15]:
def calculate_team_points(row):
    # win - 3 points
    if row['Team_score'] > row['Opp_score']:
        return 3
    # draw - 1 point
    elif row['Team_score'] == row['Opp_score']:
        return 1
    # loss - 0 points
    else:
        return 0
    
df['Team_result'] = df.apply(calculate_team_points, axis=1)

In [16]:
def add_rolling_features(df, standard_features, features_to_roll):
    features = standard_features
    
    for r in rolling_gameweeks:
        form_means = df.groupby(["Name"])[features_to_roll].rolling(r, min_periods=1).mean().groupby(["Name"]).shift(1).reset_index()
        # print(form_means[form_means["Name"] == "Erling-Haaland"])
        form_means = form_means.fillna(method='bfill') # slightly incorrect, better to drop Nan
        form_means.columns = [f'{col}{"_"}{r}' for col in form_means.columns]
        features += form_means.columns.tolist()
        features = list(filter(lambda x: x not in ["Name_" + str(r)], features))
        df = pd.concat([df.reset_index(), form_means], axis=1)
        # df = df.merge(form_means, left_index=True, right_index=True)
        df = df.drop([col for col in df.columns if col.startswith('level')], axis=1)
        
    return df.reset_index(), features

In [17]:
def ohe(df, ohe_columns, features):
    # one hot encoding
    for c in ohe_columns:
        ohe_c = pd.get_dummies(df[c], dtype="int64")
        df = pd.concat([df, ohe_c], axis=1)

        features += ohe_c.columns.tolist()
        
    return df, features

In [18]:
def label_encoding(df, column_to_encode):
    
    mapping_dict = {
        'Y': 1, # Starting eleven
        'Y*': 1, # Starting eleven as captain
        'N': 0, # Not in starting eleven
    }
    
    df[column_to_encode] = df[column_to_encode].map(mapping_dict)
    
    return df

In [19]:
df = label_encoding(df, "Start")

In [20]:
df, features = add_rolling_features(df, standard_features, features_to_roll)

In [21]:
df, features = ohe(df, ["FPL_pos"], features)
if "GK" in df.columns:
    df = df.drop("GK", axis=1)
    features.remove("GK")

In [22]:
# dropping unwanted columns
features = [col for col in features if not col.startswith('level')]
df = df[np.unique(features + info + [to_predict])]

In [23]:
df = df[pd.to_numeric(df["GW"], errors="coerce").notna()]
df["GW"] = df["GW"].astype("uint64")

In [24]:
# dropping NaNs
df = df.dropna(axis=0)

In [25]:
# df = df.set_index(["Name_original", "Season", "GW"], drop=False)

In [26]:
df.columns

Index(['Ast_2', 'Ast_30', 'Ast_4', 'FPL_points', 'FPL_points_2',
       'FPL_points_30', 'FPL_points_4', 'FPL_pos', 'GW', 'Gls_2', 'Gls_30',
       'Gls_4', 'Min_2', 'Min_30', 'Min_4', 'Name_original', 'Opp_rating',
       'Opp_score_2', 'Opp_score_30', 'Opp_score_4', 'Opponent',
       'Rating_difference', 'Season', 'Team', 'Team_CS_2', 'Team_CS_30',
       'Team_CS_4', 'Team_rating', 'Team_result_2', 'Team_result_30',
       'Team_result_4', 'Team_score_2', 'Team_score_30', 'Team_score_4',
       'Was_home', 'xA_2', 'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'xP'],
      dtype='object')

In [27]:
df.shape

(1740, 42)

In [28]:
# saving to file
df.set_index(["Name_original", "Season", "GW"], drop=False).to_csv("data/misc/df_features.csv", index=False) # keeping GKs on

In [29]:
# df = df[~df["FPL_pos"].isin(["GK"])] # only outfield players

# Data split into train and valid
Let's take entire 2021-22 season and 30 first gameweeks of 2022-23 season as training data and 8 last gameweeks of that season as valid data (~10% of all rows).

In [30]:
def shrink_df_to_top_players(df, n_players, min_fixtures):
    # Calculate average scores
    average_scores = df.groupby('Name_original')[to_predict].mean()
    
    # Calculate player counts
    player_counts = df['Name_original'].value_counts()
    
    # Filter out players with less than min_fixtures occurrences
    popular_players = player_counts[player_counts >= min_fixtures].index
    
    print("Unique players with min_fixtures:", popular_players.size)

    # Sort players by average scores
    sorted_players = average_scores.sort_values(ascending=False)

    # Get the top n_players players
    top_popular_players = sorted_players[sorted_players.index.isin(popular_players)].head(n_players)

    # Filter the original DataFrame
    top_players_df = df[df['Name_original'].isin(top_popular_players.index)]
    
    print("Unique players left:", top_players_df['Name_original'].unique().size)
    
    return top_players_df

In [31]:
CUT_OFF_GAMEWEEK = 31
SEASON_TO_PREDICT = "2022-23"

In [32]:
df_train = shrink_df_to_top_players(df, 550, 10)
# df_train["Name_original"].unique()

Unique players with min_fixtures: 37
Unique players left: 37


In [33]:
df = df.set_index(["Name_original", "Season", "GW"], drop=False)

In [34]:
# training data
X_train = df_train[((df_train["Season"] < SEASON_TO_PREDICT) | (df_train["GW"] <= CUT_OFF_GAMEWEEK - 1))].reset_index(drop=True)
y_train = df_train[((df_train["Season"] < SEASON_TO_PREDICT) | (df_train["GW"] <= CUT_OFF_GAMEWEEK - 1))].reset_index(drop=True)

In [35]:
# only 31st gameweek
X_test = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)
y_test = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)

In [36]:
# all remaining gameweeks
X_test_all_remaining = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)
y_test_all_remaining = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)

In [37]:
X_train.shape, X_test.shape, X_test_all_remaining.shape

((1520, 42), (24, 42), (206, 42))

# Model

In [38]:
class PositionalModel:
    def __init__(self):
        self.model_GK = XGBRegressor(random_state=42, verbosity = 0)
        self.model_DEF = XGBRegressor(random_state=42, verbosity = 0)
        self.model_MID = XGBRegressor(random_state=42, verbosity = 0)
        self.model_FWD = XGBRegressor(random_state=42, verbosity = 0)
        
    def fit(self, X, y):
        # print(y[y["FPL_pos"] == "GK"].shape)
        self.model_GK.fit(X[X["FPL_pos"] == "GK"][features], y[y["FPL_pos"] == "GK"][to_predict], verbose=False)
        self.model_DEF.fit(X[X["FPL_pos"] == "DEF"][features], y[y["FPL_pos"] == "DEF"][to_predict], verbose=False)
        self.model_MID.fit(X[X["FPL_pos"] == "MID"][features], y[y["FPL_pos"] == "MID"][to_predict], verbose=False)
        self.model_FWD.fit(X[X["FPL_pos"] == "FWD"][features], y[y["FPL_pos"] == "FWD"][to_predict], verbose=False)
        
    def predict(self, X):
        output = []
        for i, row in X.iterrows():
            # print(i)
            if row["FPL_pos"] == "GK":
                output.append( self.model_GK.predict( X.head(i+1).tail(1)[features] ) )
            if row["FPL_pos"] == "DEF":
                # print(row[features])
                output.append( self.model_DEF.predict( X.head(i+1).tail(1)[features] ) )
            if row["FPL_pos"] == "MID":
                output.append( self.model_MID.predict( X.head(i+1).tail(1)[features] ) )
            if row["FPL_pos"] == "FWD":
                output.append( self.model_FWD.predict( X.head(i+1).tail(1)[features] ) )
        return np.array(output).flatten()

In [39]:
# model = GradientBoostingRegressor(random_state=42)
model = PositionalModel()
# model = RandomForestRegressor(random_state=42)

In [40]:
%%time
model.fit(
    X_train,
    y_train
)

CPU times: total: 1.75 s
Wall time: 515 ms


# Getting predictions

In [41]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    if all_remaining:
        df_predictions = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT)].reset_index(drop=True)
    else:
        df_predictions = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT)].reset_index(drop=True)
    
    df_predictions.loc[:, "Pred"] = predictions
    
    preds = df_predictions[info + [to_predict] + ["Pred"]].sort_values(by=["Pred"], ascending = False)
    
    return preds, predictions

In [42]:
preds, predictions = get_predictions(model, df, X_test)

X SHAPE (24, 42)


In [43]:
# X_test.isna().sum().sum()

In [44]:
preds_all_gameweeks, predictions_all = get_predictions(model, df, X_test_all_remaining, True)

X SHAPE (206, 42)


In [45]:
df.columns

Index(['Ast_2', 'Ast_30', 'Ast_4', 'FPL_points', 'FPL_points_2',
       'FPL_points_30', 'FPL_points_4', 'FPL_pos', 'GW', 'Gls_2', 'Gls_30',
       'Gls_4', 'Min_2', 'Min_30', 'Min_4', 'Name_original', 'Opp_rating',
       'Opp_score_2', 'Opp_score_30', 'Opp_score_4', 'Opponent',
       'Rating_difference', 'Season', 'Team', 'Team_CS_2', 'Team_CS_30',
       'Team_CS_4', 'Team_rating', 'Team_result_2', 'Team_result_30',
       'Team_result_4', 'Team_score_2', 'Team_score_30', 'Team_score_4',
       'Was_home', 'xA_2', 'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'xP'],
      dtype='object')

# Custom metric
Such models are usually used to compare players and decide who to pick for upcoming gameweek(s). For each player pair in a subset of players model can either predict the outcome well (e.g. player A scores more than player B and model predicts exactly that) or predict wrongly (e.g. player A scores more than player B but model predicts player B > player A). Pairwise_accuracy returns the percentage of corrected predicted pairs. Pairwise_accuracy_topX is a variation of this metric calculated only for X highest scoring players of last two seasons.

In [46]:
def get_top_performer_names(df, no_top, no_gws):
    # takes no_top players that recorded highest average FPL points in no_gws last gameweeks
    return df.reset_index(drop=True).groupby("Name_original")[to_predict].mean().groupby("Name_original").tail(no_gws).sort_values(ascending=False).head(no_top).index.to_list()

In [47]:
def pairwise_accuracy(predicted_scores, true_scores):
    if len(predicted_scores) != len(true_scores):
        raise ValueError("The length of predicted_scores and true_scores must be the same.")

    num_pairs = 0
    num_correct_pairs = 0

    for i in range(len(predicted_scores)):
        for j in range(i + 1, len(predicted_scores)):
            # Check if the predicted order matches the true order
            if (predicted_scores[i] > predicted_scores[j] and true_scores[i] > true_scores[j]) or \
               (predicted_scores[i] < predicted_scores[j] and true_scores[i] < true_scores[j]):
                num_correct_pairs += 1
            num_pairs += 1

    pairwise_accuracy = num_correct_pairs / num_pairs
    return pairwise_accuracy

In [48]:
def pairwise_accuracy_topX(model, df, top_x, all_gw=False):
    # pairwise_accuracy for top_X players
    if all_gw:
        top_performers = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name_original"].isin(get_top_performer_names(df, top_x, 50)))]
    else:
        top_performers = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name_original"].isin(get_top_performer_names(df, top_x, 50)))]

    X = top_performers[features].reset_index(drop=True)
    y = np.array(top_performers[to_predict].reset_index(drop=True))
    preds = model.predict(X)
    
    return  pairwise_accuracy(preds, y)

# Evaluation

In [49]:
def evaluate(model, df, predictions, y_true, all_gw=False):
    # function to calculate different metrics for a given model
    mae = mean_absolute_error(y_true["FPL_points"], predictions)
    mse = mean_squared_error(y_true["FPL_points"], predictions)

    # pairwise_acc = pairwise_accuracy(y_true, predictions)
    # pairwise_accuracy_top20 = pairwise_accuracy_topX(model, df, 20, all_gw)
    # pairwise_accuracy_top100 = pairwise_accuracy_topX(model, df, 100, all_gw)
    
    print("MAE:", mae)
    print("MSE:", mse)
    # print("Pairwise accuracy:", pairwise_acc)
    # print("Pairwise accuracy @TOP100:", pairwise_accuracy_top100)
    # print("Pairwise accuracy @TOP20:", pairwise_accuracy_top20)
    print("Avg pred vs avg true:", np.mean(predictions).round(2), "vs", np.mean(y_true[to_predict]).round(2).item())

In [50]:
# df[(df["GW"] == GAMEWEEK_TO_PREDICT) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name"].isin(get_top_performer_names(df, 20, 50)))].columns

In [51]:
# next gameweek
evaluate(model, df, predictions, y_test, all_gw=False)

MAE: 1.1744880268039803
MSE: 5.0393968444538855
Avg pred vs avg true: 1.81 vs 1.83


In [52]:
# all remaining gameweeks
evaluate(model, df, predictions_all, y_test_all_remaining, all_gw=True)

MAE: 1.6316035221256224
MSE: 6.562758081023589
Avg pred vs avg true: 2.04 vs 1.76


In [53]:
# df[df["Name"] == "Abdoulaye-Doucoure"][info]

In [54]:
# model.predict(df[df["GW"] == GAMEWEEK_TO_PREDICT][df["Name"] == "Mohamed-Salah"][features])

In [55]:
features

['Was_home',
 'Rating_difference',
 'Min_2',
 'Gls_2',
 'Ast_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'FPL_points_2',
 'Min_4',
 'Gls_4',
 'Ast_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'FPL_points_4',
 'Min_30',
 'Gls_30',
 'Ast_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'FPL_points_30']

In [56]:
len(features)

32

# Predictions - next gameweek only

In [57]:
preds.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,FPL_points,xP,Pred
7,Emiliano Martínez Romero,31,2022-23,Aston Villa,Newcastle Utd,1.0,1801.513550,1876.322510,GK,6.0,6.0,5.775107
10,Jordan Pickford,31,2022-23,Everton,Fulham,1.0,1687.977173,1710.906738,GK,2.0,2.0,4.732130
18,Nick Pope,31,2022-23,Newcastle Utd,Aston Villa,0.0,1876.322510,1801.513550,GK,2.0,2.0,4.649340
5,David Raya Martin,31,2022-23,Brentford,Wolves,0.0,1782.324097,1717.951904,GK,5.0,5.0,4.519623
0,Aaron Ramsdale,31,2022-23,Arsenal,West Ham,0.0,1946.847900,1751.608521,GK,1.0,1.0,4.136261
17,Norberto Murara Neto,31,2022-23,Bournemouth,Tottenham,0.0,1665.276123,1851.721558,GK,3.0,3.0,3.476113
9,Hugo Lloris,31,2022-23,Tottenham,Bournemouth,1.0,1851.721558,1665.276123,GK,2.0,2.0,3.140235
12,Lukasz Fabianski,31,2022-23,West Ham,Arsenal,1.0,1751.608521,1946.847900,GK,2.0,2.0,2.970688
11,José Malheiro de Sá,31,2022-23,Wolves,Brentford,1.0,1717.951904,1782.324097,GK,7.0,7.0,2.949208
2,Bernd Leno,31,2022-23,Fulham,Everton,0.0,1710.906738,1687.977173,GK,3.0,3.0,2.874491


In [58]:
preds[preds["Team"] == "Manchester City"].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,FPL_points,xP,Pred
6,Ederson Santana de Moraes,31,2022-23,Manchester City,Leicester City,1.0,2060.531738,1722.042969,GK,2.0,2.0,2.050133


# Predictions - all remaining gameweeks

In [59]:
preds_all_gameweeks.head(15)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,FPL_points,xP,Pred
3,Aaron Ramsdale,34,2022-23,Arsenal,Chelsea,1.0,1921.972534,1789.972534,GK,3.0,3.0,8.914800
92,Jason Steele,35,2022-23,Brighton,Everton,1.0,1831.996948,1667.626831,GK,-2.0,-2.0,8.648385
197,Vicente Guaita,36,2022-23,Crystal Palace,Bournemouth,1.0,1744.869995,1670.303223,GK,0.0,0.0,7.430630
2,Aaron Ramsdale,33,2022-23,Arsenal,Manchester City,0.0,1928.582397,2057.953369,GK,1.0,1.0,6.902019
164,Nick Pope,34,2022-23,Newcastle Utd,Southampton,1.0,1876.238159,1634.447266,GK,2.0,2.0,6.237974
69,Ederson Santana de Moraes,37,2022-23,Manchester City,Chelsea,1.0,2087.472656,1794.311890,GK,0.0,0.0,6.160018
120,Lukasz Fabianski,37,2022-23,West Ham,Leeds United,1.0,1770.251953,1671.186279,GK,1.0,1.0,6.148499
11,Alisson Ramses Becker,35,2022-23,Liverpool,Brentford,1.0,1940.118652,1782.052979,GK,5.0,5.0,6.007432
196,Vicente Guaita,35,2022-23,Crystal Palace,Tottenham,0.0,1749.267822,1813.097046,GK,0.0,0.0,5.840761
72,Emiliano Martínez Romero,31,2022-23,Aston Villa,Newcastle Utd,1.0,1801.513550,1876.322510,GK,6.0,6.0,5.775107


In [60]:
preds_all_gameweeks[preds_all_gameweeks["Name_original"] == "Mohamed Salah"].sort_values(by=["GW"])

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,FPL_points,xP,Pred


In [61]:
preds_all_gameweeks[preds_all_gameweeks["Name_original"] == "Erling Haaland"].sort_values(by=["GW"])

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,FPL_points,xP,Pred


# Saving to file

In [62]:
preds_all_gameweeks.to_csv("predictions/all_predictions.csv", index=False)

In [63]:
pickle.dump(model, open("models/GBR.pkl", 'wb'))

# Feature importance and influence

In [64]:
# explainer = shap.Explainer(model.predict, X_test_all_remaining)
# shap_values = explainer(X_test_all_remaining)

In [65]:
# shap.plots.bar(shap_values, max_display=15)

In [66]:
# shap.plots.beeswarm(shap_values, max_display=15)

In [67]:
# # explaining Erling Haaland`s score in gameweek 31
# shap.plots.bar(shap_values[69], max_display=15)

In [68]:
# # Salah vs Nottingham Forest
# shap.plots.waterfall(shap_values[1396])